In [1]:
import numpy as np
import pickle as pk
import openpnm as op
import matplotlib.pyplot as plt
np.random.seed(10)

# Introduction
This example is from our series of permeability benchmarks, mostly for making sure that `OpenPNM` can accurately predict the permeability of known rock samples. In this example, we try to calculate the permeability of an extracted pore network model form a Bentheimer sandstone rock sample. The network has already been extracted using `PoreSpy`, and so here we only load the output generated from `PoreSpy` into `OpenPNM`.

To load extracted networks from `PoreSpy` into `OpenPNM`, it's highly recommended to use `OpenPNM`'s exclusive IO class for `PoreSpy`. Here's how it works: (basically the IO class returns a project object)

In [2]:
fname = '../../fixtures/Extracted-Networks/Berea'
prj = op.io.PoreSpy.load(fname)
net = prj.network

In addition to assigning a network, the `PoreSpy` IO class automatically assigns a geometry using the loaded data as well. To fetch the geometry, we can do that by asking the project object, but we need to know the geometry name. If nothing is provided by the user, `OpenPNM` usually automatically assign names to objects in a sequential manner. For instance, the first assigned network or geometry is named "net_01", "geo_01", respectively. But let's say you didn't know about this. Let's ispect the project object and see what geometries are attached to it:

In [3]:
#NBVAL_IGNORE_OUTPUT
print(prj.geometries())

{'geo_01': <openpnm.geometry.Imported object at 0x1f44feb7f40>}


Great! Now, we know that there's only a singly geometry attached to the project object, which can be accessed via the key "geo_01". Let's fetch the geometry:

In [4]:
geo = prj.geometries()["geo_01"]

Now, let's take a look at the network using `ParaView`:

<img src="https://user-images.githubusercontent.com/14086031/79818827-b24c6a80-8356-11ea-9eaf-81b9deb79cda.png" style="width: 65%" align="left"/>

# Pre-processing
Now, let's run `StokesFlow` algorithm on this network to calculate its permeability. Usually, in the process of network extraction, some isolated pores could exist in the final network. Having isolated pores in your network rightfully confuses `OpenPNM` as gives rise to `NaNs` in your solution. We need to get rid of those pores. To do that, first we call `check_network_health` on the network, which returns back a dictionary containing useful information regarding network health. For now, we're only interested in the the "trim_pores" key in that dictionary, which contains a list of pores that need to be trimmed off of the network in order for the pores to be fully connected. Here's how we do this:

In [5]:
# Look for disconnected pores and trim them
net_health = net.check_network_health()
op.topotools.trim(network=net, pores=net_health["trim_pores"])

# Permeability calculation
Now, let's create our `StokesFlow` algorithm object, impose a constant pressure drop across the y-axis, and calculate the y-axis permeability based on the resulting flow rate.

Note that first things first! For `OpenPNM` to be able to run any algorithm, we need to define a `phase` object and a `physics` object. More precisely, you need as many `physics` objects as your `geometry` objects. For instance, if your network consists of  3 geometries, you need to define 3 separate `physics` objects. Here, since there's only one `geometry` object, we only need to create 1 `physics` object.

In [6]:
# Add phase and physics
air = op.phases.Air(network=net)
phys_air = op.physics.Standard(network=net, phase=air, geometry=geo)

# Perform StokesFlow algorithm to calculate permeability
perm = op.algorithms.StokesFlow(network=net)
perm.setup(phase=air)
perm.set_value_BC(pores=net.pores('left'), values=0)
perm.set_value_BC(pores=net.pores('right'), values=101325)
perm.run()
air.update(perm.results())

# Original image shape (tomography data) needed to calculate cross-sectional area and length
shape = np.array([1024, 1024, 1024])
voxel_size = 2.7745e-6
area = shape[[0, 2]].prod() * voxel_size**2
length = shape[1] * voxel_size
K = perm.calc_effective_permeability(domain_area=area, domain_length=length)
# 1e12 is the conversion factor to convert from SI to darcy units
print(f"Permeability (Berea sandstone): {K[0]*1e12:.4f} Darcy")

Permeability (Berea sandstone): 1.1769 Darcy
